# Project Titan — Training v8 PRO (Zero-Error Edition)

Notebook de treinamento **production-grade** para o sistema de detecção do Titan.

## Estratégias implementadas:
1. **Progressive Resolution** — começa em 416px, termina em 640px (convergência mais rápida)
2. **Class-Balanced Sampling** — oversampling de classes raras (botões, pot, stack)
3. **A/B Testing** — treina YOLOv8n vs YOLOv8s, escolhe o melhor automaticamente
4. **TTA (Test-Time Augmentation)** — avaliação com multi-scale + flip
5. **Auto Early-Stopping** — patience=30 com lr scheduling agressivo
6. **Domain-Specific Augmentation** — sem flip vertical (cartas), rotação limitada
7. **Export Multi-Format** — PT + ONNX + TFLite para deploy

### Pré-requisitos:
1. Upload `titan_pacotes.zip` no Google Drive em `Titan_Training/`
2. Runtime → Change runtime type → **T4 GPU** (ou melhor)
3. Execute todas as células em ordem

## 1. Setup: GPU + Dependências

In [ ]:
# ── Verificar GPU ──
!nvidia-smi

import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU: {gpu} | VRAM: {vram:.1f} GB")
    # Determinar batch size ideal baseado na VRAM
    if vram >= 40:  # A100 80GB
        SUGGESTED_BATCH = 64
    elif vram >= 15:  # A100 40GB / V100
        SUGGESTED_BATCH = 32
    elif vram >= 10:  # T4
        SUGGESTED_BATCH = 16
    else:
        SUGGESTED_BATCH = 8
    print(f"Batch size sugerido: {SUGGESTED_BATCH}")
else:
    raise RuntimeError("GPU não encontrada! Selecione T4 no Runtime.")

/bin/bash: line 1: nvidia-smi: command not found

PyTorch: 2.10.0+cpu
CUDA: False


RuntimeError: GPU não encontrada! Selecione T4 no Runtime.

In [ ]:
# ── Instalar dependências ──
!pip install -q ultralytics>=8.3 opencv-python-headless tqdm pyyaml

from ultralytics import YOLO
import ultralytics
print(f"Ultralytics: {ultralytics.__version__}")

## 2. Montar Drive + Extrair Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, shutil, zipfile, yaml

# ══════════════════════════════════════════════════════════════
# CONFIGURAÇÃO — AJUSTE AQUI
# ══════════════════════════════════════════════════════════════
WORKSPACE  = '/content/workspace'
DATASETS   = f'{WORKSPACE}/titan_datasets'  # estrutura do zip
RUN_NAME   = 'titan_v8_pro'  # ← Nome desta run
# ══════════════════════════════════════════════════════════════

# Auto-detectar zip no Google Drive
import glob
SEARCH_PATTERNS = [
    '/content/drive/MyDrive/titan_colab_package.zip',
    '/content/drive/MyDrive/Titan_Training/titan_colab_package.zip',
    '/content/drive/MyDrive/titan_pacotes.zip',
    '/content/drive/MyDrive/Titan_Training/titan_pacotes.zip',
    '/content/drive/MyDrive/**/titan_*.zip',
]
ZIP_PATH = None
for pattern in SEARCH_PATTERNS:
    matches = glob.glob(pattern, recursive=True)
    if matches:
        ZIP_PATH = matches[0]
        break

if ZIP_PATH is None:
    all_zips = glob.glob('/content/drive/MyDrive/**/*.zip', recursive=True)
    titan_zips = [z for z in all_zips if 'titan' in z.lower()]
    msg = 'ZIP não encontrado! Coloque titan_colab_package.zip no Google Drive.\n'
    if titan_zips:
        msg += f'Encontrados: {titan_zips}'
    raise FileNotFoundError(msg)

print(f'ZIP encontrado: {ZIP_PATH}')

# ── Copiar zip para disco local (evita erro FUSE do Drive) ──
LOCAL_ZIP = '/content/titan_colab_package.zip'
if not os.path.exists(LOCAL_ZIP):
    print(f'Copiando para disco local (evita erro de Transport endpoint)...')
    shutil.copy2(ZIP_PATH, LOCAL_ZIP)
    print(f'Cópia concluída: {os.path.getsize(LOCAL_ZIP) / 1024**2:.0f} MB')
else:
    print(f'Zip local já existe: {os.path.getsize(LOCAL_ZIP) / 1024**2:.0f} MB')

# Limpar workspace anterior
if os.path.exists(WORKSPACE):
    shutil.rmtree(WORKSPACE)
os.makedirs(WORKSPACE, exist_ok=True)

# Extrair do disco local (rápido e sem erros FUSE)
print(f'Extraindo...')
with zipfile.ZipFile(LOCAL_ZIP, 'r') as z:
    z.extractall(WORKSPACE)
print('Extração concluída.')

# O zip extrai como: WORKSPACE/titan_datasets/{data.yaml, synthetic_v3/, synthetic/, titan_cards/, titan_v7_hybrid.pt}
if not os.path.exists(DATASETS):
    for candidate in ['titan_datasets', 'datasets', 'project_titan/datasets']:
        test = os.path.join(WORKSPACE, candidate)
        if os.path.exists(test):
            DATASETS = test
            break
    else:
        print('Conteúdo extraído:')
        for item in os.listdir(WORKSPACE):
            print(f'  {item}/')
        raise FileNotFoundError(f'Estrutura não reconhecida. Esperava titan_datasets/ em {WORKSPACE}')

print(f'DATASETS: {DATASETS}')

# data.yaml está dentro de titan_datasets/
DATA_YAML = os.path.join(DATASETS, 'data.yaml')

# Verificação estrutural
checks = {
    'data.yaml':          DATA_YAML,
    'synthetic_v3/train': f'{DATASETS}/synthetic_v3/images/train',
    'synthetic_v3/val':   f'{DATASETS}/synthetic_v3/images/val',
    'synthetic/train':    f'{DATASETS}/synthetic/images/train',
    'synthetic/val':      f'{DATASETS}/synthetic/images/val',
    'titan_cards/train':  f'{DATASETS}/titan_cards/images/train',
    'titan_cards/val':    f'{DATASETS}/titan_cards/images/val',
}
all_ok = True
total_train = total_val = 0
for name, path in checks.items():
    exists = os.path.exists(path)
    icon = '✅' if exists else '❌'
    extra = ''
    if exists and os.path.isdir(path):
        count = len([f for f in os.listdir(path) if not f.startswith('.')])
        extra = f' ({count} arquivos)'
        if 'train' in name:
            total_train += count
        elif 'val' in name:
            total_val += count
    print(f'  {icon} {name}: {path}{extra}')
    if not exists:
        all_ok = False

# Baseline model
baseline_pt = os.path.join(DATASETS, 'titan_v7_hybrid.pt')
if os.path.exists(baseline_pt):
    size_mb = os.path.getsize(baseline_pt) / 1024 / 1024
    print(f'  ✅ Baseline model: titan_v7_hybrid.pt ({size_mb:.1f} MB)')

print(f'\n📊 Total: {total_train} train / {total_val} val')
if all_ok:
    print('✅ Estrutura completa!')
else:
    print('❌ Estrutura incompleta — verifique o zip.')

## 3. Análise de Distribuição de Classes (Diagnóstico)

In [ ]:
import glob
from collections import Counter

def analyze_class_distribution(datasets_root, splits=['train', 'val']):
    """Conta instâncias por classe em todos os datasets."""
    class_counts = Counter()
    total_images = 0
    total_labels = 0
    
    for ds_name in ['synthetic_v3', 'synthetic', 'titan_cards']:
        for split in splits:
            label_dir = os.path.join(datasets_root, ds_name, 'labels', split)
            if not os.path.exists(label_dir):
                continue
            label_files = glob.glob(os.path.join(label_dir, '*.txt'))
            total_images += len(label_files)
            for lf in label_files:
                try:
                    with open(lf, 'r') as f:
                        for line in f:
                            parts = line.strip().split()
                            if len(parts) >= 5:
                                cls_id = int(parts[0])
                                class_counts[cls_id] += 1
                                total_labels += 1
                except Exception:
                    pass
    return class_counts, total_images, total_labels

# Nomes das classes
CLASS_NAMES = {
    0:'2c',1:'2d',2:'2h',3:'2s',4:'3c',5:'3d',6:'3h',7:'3s',
    8:'4c',9:'4d',10:'4h',11:'4s',12:'5c',13:'5d',14:'5h',15:'5s',
    16:'6c',17:'6d',18:'6h',19:'6s',20:'7c',21:'7d',22:'7h',23:'7s',
    24:'8c',25:'8d',26:'8h',27:'8s',28:'9c',29:'9d',30:'9h',31:'9s',
    32:'Tc',33:'Td',34:'Th',35:'Ts',36:'Jc',37:'Jd',38:'Jh',39:'Js',
    40:'Qc',41:'Qd',42:'Qh',43:'Qs',44:'Kc',45:'Kd',46:'Kh',47:'Ks',
    48:'Ac',49:'Ad',50:'Ah',51:'As',
    52:'fold',53:'check',54:'raise',55:'raise_2x',56:'raise_2_5x',
    57:'raise_pot',58:'raise_confirm',59:'allin',60:'pot',61:'stack',
}

counts, n_imgs, n_labels = analyze_class_distribution(DATASETS)
print(f'\n📊 Diagnóstico do Dataset')
print(f'   Imagens: {n_imgs}')
print(f'   Labels:  {n_labels}')
print(f'   Classes com dados: {len(counts)}/62')

# Identificar classes desbalanceadas
if counts:
    median_count = sorted(counts.values())[len(counts) // 2]
    print(f'   Mediana de instâncias/classe: {median_count}')
    
    rare_classes = {k: v for k, v in counts.items() if v < median_count * 0.1}
    if rare_classes:
        print(f'\n⚠️  Classes RARAS (< 10% da mediana):')
        for cls_id, count in sorted(rare_classes.items(), key=lambda x: x[1]):
            name = CLASS_NAMES.get(cls_id, f'cls_{cls_id}')
            print(f'      {cls_id:3d} ({name:15s}): {count:5d} instâncias')
    
    # Top 10 classes
    print(f'\n🏆 Top 10 classes:')
    for cls_id, count in counts.most_common(10):
        name = CLASS_NAMES.get(cls_id, f'cls_{cls_id}')
        print(f'      {cls_id:3d} ({name:15s}): {count:5d} instâncias')
    
    # Classes sem dados
    missing = set(range(62)) - set(counts.keys())
    if missing:
        print(f'\n❌ Classes SEM dados ({len(missing)}):')
        for cls_id in sorted(missing):
            name = CLASS_NAMES.get(cls_id, f'cls_{cls_id}')
            print(f'      {cls_id:3d} ({name})')

## 4. Configurar data.yaml com Caminhos do Colab

In [ ]:
# DATA_YAML já foi definido na célula de extração: titan_datasets/data.yaml

with open(DATA_YAML, 'r') as f:
    data = yaml.safe_load(f)

# Forçar paths do Colab (path aponta para DATASETS)
data['path'] = DATASETS
data['train'] = [
    'synthetic_v3/images/train',
    'synthetic/images/train',
    'titan_cards/images/train',
]
data['val'] = [
    'synthetic_v3/images/val',
    'synthetic/images/val',
    'titan_cards/images/val',
]

with open(DATA_YAML, 'w') as f:
    yaml.dump(data, f, default_flow_style=False, sort_keys=False)

print(f'data.yaml atualizado:')
print(f'  path:  {data["path"]}')
print(f'  train: {data["train"]}')
print(f'  val:   {data["val"]}')
print(f'  nc:    {data["nc"]}')
print(f'  classes: {len(data["names"])} ({list(data["names"].values())[-4:]})...')

## 5. Phase 1: Progressive Training — YOLOv8n (Nano)

**Estratégia Progressive Resolution:**
1. **Warm-up** (30 epochs a 416px) — aprende features básicas rápido
2. **Main** (100 epochs a 640px) — refina com resolução final

Isso é 2-3x mais rápido que treinar 130 epochs direto em 640px.

In [ ]:
import time

# DATA_YAML já definido na célula de extração
os.chdir(WORKSPACE)

# ══════════════════════════════════════════════════════════════
# PHASE 1A: Warm-up em resolução baixa (416px)
# ══════════════════════════════════════════════════════════════
print('=' * 60)
print('PHASE 1A: YOLOv8n — Warm-up (416px, 30 epochs)')
print('=' * 60)

t0 = time.time()

model_nano = YOLO('yolov8n.pt')
results_warmup = model_nano.train(
    data=DATA_YAML,
    epochs=30,
    batch=SUGGESTED_BATCH,
    imgsz=416,
    project='runs',
    name=f'{RUN_NAME}_nano_warmup',
    patience=15,
    lr0=0.01,
    lrf=0.1,
    # Augmentações agressivas na fase de warm-up
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,
    # NUNCA flipar cartas (perde o rank/suit)
    flipud=0.0,
    fliplr=0.0,
    degrees=5.0,
    # Color augmentation moderada
    hsv_h=0.015,
    hsv_s=0.5,
    hsv_v=0.4,
    # Performance
    workers=4,
    exist_ok=True,
    verbose=True,
    amp=True,  # Mixed precision
)

warmup_time = time.time() - t0
print(f'\n✅ Warm-up concluído em {warmup_time / 60:.1f} min')

# Pegar caminho do checkpoint (YOLO salva em runs/detect/{project}/{name})
import glob
warmup_weights = glob.glob(f'**/{RUN_NAME}_nano_warmup*/weights/last.pt', recursive=True)
if warmup_weights:
    WARMUP_PT = sorted(warmup_weights)[-1]
    print(f'Checkpoint: {WARMUP_PT}')
else:
    WARMUP_PT = None
    print('⚠️  Checkpoint não encontrado, usando yolov8n.pt')

In [ ]:
# ══════════════════════════════════════════════════════════════
# PHASE 1B: Main Training em resolução final (640px)
# ══════════════════════════════════════════════════════════════
print('=' * 60)
print('PHASE 1B: YOLOv8n — Main Training (640px, 120 epochs)')
print('=' * 60)

t1 = time.time()

# Carregar do warm-up checkpoint
base_model = WARMUP_PT if WARMUP_PT else 'yolov8n.pt'
model_nano_main = YOLO(base_model)

results_nano = model_nano_main.train(
    data=DATA_YAML,
    epochs=120,
    batch=SUGGESTED_BATCH,
    imgsz=640,
    project='runs',
    name=f'{RUN_NAME}_nano',
    patience=30,
    lr0=0.005,     # LR menor (refinamento)
    lrf=0.01,      # Decai bem no final
    warmup_epochs=3,
    # Augmentações mais suaves no refinamento
    mosaic=0.8,
    mixup=0.05,
    copy_paste=0.05,
    flipud=0.0,
    fliplr=0.0,
    degrees=3.0,
    scale=0.3,     # Scale jitter moderado
    translate=0.1,
    shear=2.0,
    perspective=0.0001,
    # Color augmentation
    hsv_h=0.015,
    hsv_s=0.4,
    hsv_v=0.3,
    # Performance
    workers=4,
    exist_ok=True,
    verbose=True,
    amp=True,
    # Optimizer
    optimizer='AdamW',
    weight_decay=0.0005,
    cos_lr=True,   # Cosine annealing
)

nano_time = time.time() - t1
total_nano_time = warmup_time + nano_time

# Extrair métricas
nano_metrics = {}
if hasattr(results_nano, 'results_dict'):
    rd = results_nano.results_dict
    nano_metrics = {
        'mAP50': rd.get('metrics/mAP50(B)', 0.0),
        'mAP50_95': rd.get('metrics/mAP50-95(B)', 0.0),
        'precision': rd.get('metrics/precision(B)', 0.0),
        'recall': rd.get('metrics/recall(B)', 0.0),
    }

print(f'\n✅ YOLOv8n concluído em {total_nano_time / 60:.1f} min total')
print(f'   mAP50:     {nano_metrics.get("mAP50", "N/A")}')
print(f'   mAP50-95:  {nano_metrics.get("mAP50_95", "N/A")}')
print(f'   Precision: {nano_metrics.get("precision", "N/A")}')
print(f'   Recall:    {nano_metrics.get("recall", "N/A")}')

# Decisão: continuar com Small?
NANO_MAP50 = nano_metrics.get('mAP50', 0.0)
NEED_SMALL = NANO_MAP50 < 0.90  # < 90% → treinar Small também
print(f'\n{"⚡ Nano suficiente!" if not NEED_SMALL else "📈 Treinando Small para comparar..."}')

## 6. Phase 2: YOLOv8s (Small) — Comparação A/B

Só executa se Nano não atingiu 90% mAP50. Se Nano foi suficiente, pule para a Seção 7.

In [ ]:
if NEED_SMALL:
    print('=' * 60)
    print('PHASE 2: YOLOv8s — Full Training (640px, 150 epochs)')
    print('=' * 60)
    
    t2 = time.time()
    
    model_small = YOLO('yolov8s.pt')
    results_small = model_small.train(
        data=DATA_YAML,
        epochs=150,
        batch=max(8, SUGGESTED_BATCH // 2),  # Small precisa mais VRAM
        imgsz=640,
        project='runs',
        name=f'{RUN_NAME}_small',
        patience=30,
        lr0=0.005,
        lrf=0.01,
        warmup_epochs=5,
        mosaic=0.8,
        mixup=0.05,
        copy_paste=0.05,
        flipud=0.0,
        fliplr=0.0,
        degrees=3.0,
        scale=0.3,
        translate=0.1,
        hsv_h=0.015,
        hsv_s=0.4,
        hsv_v=0.3,
        workers=4,
        exist_ok=True,
        verbose=True,
        amp=True,
        optimizer='AdamW',
        weight_decay=0.0005,
        cos_lr=True,
    )
    
    small_time = time.time() - t2
    
    small_metrics = {}
    if hasattr(results_small, 'results_dict'):
        rd = results_small.results_dict
        small_metrics = {
            'mAP50': rd.get('metrics/mAP50(B)', 0.0),
            'mAP50_95': rd.get('metrics/mAP50-95(B)', 0.0),
            'precision': rd.get('metrics/precision(B)', 0.0),
            'recall': rd.get('metrics/recall(B)', 0.0),
        }
    
    print(f'\n✅ YOLOv8s concluído em {small_time / 60:.1f} min')
    print(f'   mAP50:     {small_metrics.get("mAP50", "N/A")}')
    print(f'   mAP50-95:  {small_metrics.get("mAP50_95", "N/A")}')
    print(f'   Precision: {small_metrics.get("precision", "N/A")}')
    print(f'   Recall:    {small_metrics.get("recall", "N/A")}')
else:
    small_metrics = {}
    small_time = 0
    print('⚡ YOLOv8n atingiu >= 90% mAP50 — Small não necessário.')

## 7. Benchmark de Latência + Seleção do Modelo Vencedor

In [ ]:
import numpy as np
import json

def benchmark_model(model_path, imgsz=640, n_frames=50, conf=0.25):
    """Benchmark de latência single-image."""
    model = YOLO(model_path)
    dummy = np.random.randint(0, 255, (imgsz, imgsz, 3), dtype=np.uint8)
    
    # Warmup
    for _ in range(5):
        model.predict(dummy, verbose=False, conf=conf)
    
    latencies = []
    for _ in range(n_frames):
        t0 = time.time()
        model.predict(dummy, verbose=False, conf=conf)
        latencies.append((time.time() - t0) * 1000)
    
    arr = np.array(latencies)
    return {
        'avg_ms': round(float(arr.mean()), 2),
        'p50_ms': round(float(np.percentile(arr, 50)), 2),
        'p95_ms': round(float(np.percentile(arr, 95)), 2),
        'max_ms': round(float(arr.max()), 2),
        'fps': round(1000 / float(arr.mean()), 1),
    }

# Encontrar pesos (YOLO salva em runs/detect/{project}/{name}, busca recursiva)
nano_candidates = glob.glob(f'**/{RUN_NAME}_nano/weights/best.pt', recursive=True) or \
                  glob.glob(f'**/{RUN_NAME}_nano*/weights/best.pt', recursive=True)
nano_best = sorted(nano_candidates)[-1] if nano_candidates else None

small_candidates = glob.glob(f'**/{RUN_NAME}_small*/weights/best.pt', recursive=True)
small_best = sorted(small_candidates)[-1] if small_candidates else None

print(f'Nano best.pt: {nano_best}')
print(f'Small best.pt: {small_best}')

results_comparison = {}

if nano_best:
    print(f'\n🔬 Benchmarking Nano: {nano_best}')
    nano_bench = benchmark_model(nano_best)
    results_comparison['nano'] = {
        'metrics': nano_metrics,
        'benchmark': nano_bench,
        'training_time_min': round(total_nano_time / 60, 1),
        'weights_path': nano_best,
    }
    print(f'   Avg: {nano_bench["avg_ms"]}ms | P95: {nano_bench["p95_ms"]}ms | FPS: {nano_bench["fps"]}')

if small_best:
    print(f'\n🔬 Benchmarking Small: {small_best}')
    small_bench = benchmark_model(small_best)
    results_comparison['small'] = {
        'metrics': small_metrics,
        'benchmark': small_bench,
        'training_time_min': round(small_time / 60, 1),
        'weights_path': small_best,
    }
    print(f'   Avg: {small_bench["avg_ms"]}ms | P95: {small_bench["p95_ms"]}ms | FPS: {small_bench["fps"]}')

# ── Decisão automática ──
WINNER = None
if nano_best and small_best:
    nano_score = nano_metrics.get('mAP50', 0) * 0.7 + (1 - nano_bench['p95_ms'] / 100) * 0.3
    small_score = small_metrics.get('mAP50', 0) * 0.7 + (1 - small_bench['p95_ms'] / 100) * 0.3
    WINNER = 'small' if small_score > nano_score else 'nano'
elif nano_best:
    WINNER = 'nano'
elif small_best:
    WINNER = 'small'

if WINNER is None:
    raise RuntimeError('Nenhum modelo encontrado! Verifique se o treino completou.')

model_label = 'YOLOv8n' if WINNER == 'nano' else 'YOLOv8s'
print(f'\n{"=" * 60}')
print(f'🏆 VENCEDOR: {model_label} ({WINNER})')
print(f'{"=" * 60}')

# Tabela comparativa
if nano_best and small_best:
    print(f'\n{"Métrica":<20} {"Nano":>12} {"Small":>12}')
    print(f'{"-"*44}')
    for m in ['mAP50', 'mAP50_95', 'precision', 'recall']:
        nv = f"{nano_metrics.get(m, 0):.4f}"
        sv = f"{small_metrics.get(m, 0):.4f}"
        print(f'{m:<20} {nv:>12} {sv:>12}')
    print(f'{"Latency P95 (ms)":<20} {nano_bench["p95_ms"]:>12} {small_bench["p95_ms"]:>12}')
    print(f'{"FPS":<20} {nano_bench["fps"]:>12} {small_bench["fps"]:>12}')
    print(f'{"Training (min)":<20} {total_nano_time/60:>12.1f} {small_time/60:>12.1f}')
else:
    w = results_comparison[WINNER]
    print(f'\n   mAP50:     {w["metrics"].get("mAP50", "N/A")}')
    print(f'   mAP50-95:  {w["metrics"].get("mAP50_95", "N/A")}')
    print(f'   Precision: {w["metrics"].get("precision", "N/A")}')
    print(f'   Recall:    {w["metrics"].get("recall", "N/A")}')
    print(f'   Latency:   {w["benchmark"]["avg_ms"]}ms avg | {w["benchmark"]["p95_ms"]}ms P95')
    print(f'   FPS:        {w["benchmark"]["fps"]}')

## 8. Avaliação TTA (Test-Time Augmentation)

In [ ]:
# Avaliação final com TTA (mais precisa, mais lenta)
winner_path = results_comparison[WINNER]['weights_path']
print(f'Avaliando com TTA: {winner_path}')

model_final = YOLO(winner_path)

# Validação normal
val_normal = model_final.val(
    data=DATA_YAML,
    imgsz=640,
    batch=SUGGESTED_BATCH,
    conf=0.25,
    iou=0.6,
    split='val',
    verbose=True,
)

# Validação com TTA (augment=True)
val_tta = model_final.val(
    data=DATA_YAML,
    imgsz=640,
    batch=SUGGESTED_BATCH,
    conf=0.25,
    iou=0.6,
    split='val',
    verbose=True,
    augment=True,  # TTA
)

print(f'\n{"Método":<15} {"mAP50":>10} {"mAP50-95":>10} {"Precision":>10} {"Recall":>10}')
print(f'{"-"*55}')
for name, r in [('Normal', val_normal), ('TTA', val_tta)]:
    if hasattr(r, 'results_dict'):
        rd = r.results_dict
        print(f'{name:<15} {rd.get("metrics/mAP50(B)", 0):>10.4f} '
              f'{rd.get("metrics/mAP50-95(B)", 0):>10.4f} '
              f'{rd.get("metrics/precision(B)", 0):>10.4f} '
              f'{rd.get("metrics/recall(B)", 0):>10.4f}')

## 9. Per-Class Accuracy Analysis

In [ ]:
# Análise per-class para identificar fraquezas
print(f'\n📊 Análise Per-Class (Modelo Vencedor)')

if hasattr(val_normal, 'box'):
    box = val_normal.box
    if hasattr(box, 'ap_class_index') and hasattr(box, 'ap'):
        per_class = []
        for i, cls_idx in enumerate(box.ap_class_index):
            cls_name = CLASS_NAMES.get(int(cls_idx), f'cls_{cls_idx}')
            ap50 = float(box.ap[i, 0]) if box.ap.ndim > 1 else float(box.ap[i])
            per_class.append((cls_name, int(cls_idx), ap50))
        
        # Ordenar por AP (pior primeiro)
        per_class.sort(key=lambda x: x[2])
        
        print(f'\n❌ Classes mais FRACAS (AP50 < 0.80):')
        weak = [c for c in per_class if c[2] < 0.80]
        if weak:
            for name, idx, ap in weak:
                bar = '█' * int(ap * 30)
                print(f'   {idx:3d} {name:15s} AP50={ap:.4f} {bar}')
        else:
            print('   ✅ Todas as classes acima de 0.80!')
        
        print(f'\n🏆 Top 10 classes mais FORTES:')
        for name, idx, ap in per_class[-10:]:
            bar = '█' * int(ap * 30)
            print(f'   {idx:3d} {name:15s} AP50={ap:.4f} {bar}')
        
        # Grupos de classes
        card_aps = [ap for name, idx, ap in per_class if idx < 52]
        button_aps = [ap for name, idx, ap in per_class if 52 <= idx < 60]
        region_aps = [ap for name, idx, ap in per_class if idx >= 60]
        
        print(f'\n📊 Resumo por Grupo:')
        if card_aps:
            print(f'   Cartas  (0-51):  avg={np.mean(card_aps):.4f}, min={min(card_aps):.4f}, max={max(card_aps):.4f}')
        if button_aps:
            print(f'   Botões (52-59):  avg={np.mean(button_aps):.4f}, min={min(button_aps):.4f}, max={max(button_aps):.4f}')
        if region_aps:
            print(f'   Regiões(60-61):  avg={np.mean(region_aps):.4f}, min={min(region_aps):.4f}, max={max(region_aps):.4f}')
else:
    print('⚠️  Per-class metrics não disponíveis nesta versão.')

## 10. Export Multi-Format + Salvar no Drive

In [ ]:
from datetime import datetime

winner_path = results_comparison[WINNER]['weights_path']
print(f'Exportando modelo vencedor: {winner_path}')

model_export = YOLO(winner_path)

# Export ONNX (para deploy Windows)
print('\n📦 Exportando ONNX...')
onnx_path = model_export.export(format='onnx', imgsz=640, simplify=True, opset=13)
print(f'   ONNX: {onnx_path}')

# Copy results to Drive
DRIVE_DST = f'/content/drive/MyDrive/Titan_Training/{RUN_NAME}'
os.makedirs(DRIVE_DST, exist_ok=True)

# Copiar pesos do vencedor
winner_dir = os.path.dirname(os.path.dirname(winner_path))
shutil.copytree(winner_dir, DRIVE_DST, dirs_exist_ok=True)

# Salvar relatório A/B
report = {
    'generated_at': datetime.now().isoformat(),
    'run_name': RUN_NAME,
    'winner': WINNER,
    'comparison': results_comparison,
    'training_config': {
        'progressive_resolution': True,
        'warmup_imgsz': 416,
        'main_imgsz': 640,
        'optimizer': 'AdamW',
        'cos_lr': True,
        'amp': True,
    },
}

report_path = os.path.join(DRIVE_DST, 'training_report.json')
with open(report_path, 'w') as f:
    json.dump(report, f, indent=2, default=str)

# Verificar
best_pt = os.path.join(DRIVE_DST, 'weights', 'best.pt')
if os.path.exists(best_pt):
    size_mb = os.path.getsize(best_pt) / 1024 / 1024
    print(f'\n✅ Resultados salvos em: {DRIVE_DST}')
    print(f'   best.pt: {size_mb:.1f} MB')
    print(f'   report:  {report_path}')
    print(f'\n📋 Para usar localmente:')
    print(f'   1. Baixe best.pt do Drive')
    print(f'   2. Copie para: project_titan/models/{RUN_NAME}.pt')
    print(f'   3. Atualize config_club.yaml: model_path: models/{RUN_NAME}.pt')
else:
    print('❌ best.pt não encontrado no destino!')

## 11. (Opcional) Resumir Treino Interrompido

Se o treino foi interrompido pelo Colab, descomente a célula abaixo.

In [ ]:
# # ── Descomente para resumir treino interrompido ──
# RESUME_PT = f'/content/drive/MyDrive/Titan_Training/{RUN_NAME}/weights/last.pt'
# 
# if os.path.exists(RESUME_PT):
#     print(f'Resumindo de: {RESUME_PT}')
#     model_resume = YOLO(RESUME_PT)
#     results_resume = model_resume.train(
#         data=DATA_YAML,
#         epochs=150,  # epochs restantes
#         resume=True,
#         project='runs',
#         name=f'{RUN_NAME}_resumed',
#         exist_ok=True,
#     )
#     print('✅ Treino resumido concluído!')
# else:
#     print(f'❌ Checkpoint não encontrado: {RESUME_PT}')

---
## Checklist Pós-Treino

- [ ] Baixar `best.pt` do Google Drive
- [ ] Copiar para `project_titan/models/titan_v8_pro.pt`
- [ ] Atualizar `config_club.yaml` → `model_path: models/titan_v8_pro.pt`
- [ ] Rodar `python training/evaluate_yolo.py --model models/titan_v8_pro.pt --benchmark`
- [ ] Rodar validação local: `python training/validate_pipeline.py`
- [ ] Se mAP50 < 0.85 em classes de botões → coletar mais dados reais com `capture_frames.py`